## Step 1 - Run this query against your SQL instance to create a database and procedure for use in the stress test

In [14]:
USE master;
GO
DROP DATABASE IF EXISTS ChicagoWhiteSox;
GO
CREATE DATABASE ChicagoWhiteSox;
GO
USE ChicagoWhiteSox;
GO
CREATE OR ALTER PROCEDURE letsgosox
AS
CREATE TABLE #gosox (col1 INT);
GO

Commands completed successfully.

Commands completed successfully.

Commands completed successfully.

Commands completed successfully.

Commands completed successfully.

Total execution time: 00:00:01.177

## Step 2 - Disable in-memory optimization for tempdb by running disableopttempdb.cmd
### The file contains the following code:
### Change the user name, password, and server name  in the cmd file as required
`sqlcmd -Usa -PJu@nS0t022 -i disableopttempdb.sql -S "localhost,1445"
docker stop sqlopttempdb
docker start sqlopttempdb`
### Once changes are made, run the following code from the command line:
`.\disableopttempdb.cmd`

## Step 2 - Execute tempstress.cmd from the command line in the directory that contains the tempdb scripts
### The file contains the following code:
### Change the user name, password, and database as needed
### When ostress completes, note the elapsed time
### You will find it in the command window, as follows: OSTRESS exiting normally, elapsed time: 00:01:08.348
`.\ostress -Usa -PJu@nS0t022 -Q"exec letsgosox" -n50 -r5000 -dChicagoWhiteSox -S"localhost,1445"`
### Once changes are made, run the following code from the command line:
`.\tempstress.cmd`

## Step 3 - While the ostress utility is running run the following query
### This will return the pages that that are being latched
### This will result in PAGELATCH_XX waits

In [15]:
USE tempdb;
GO
SELECT object_name(page_info.object_id) AS object_name, d.wait_type, page_info.* 
FROM sys.dm_exec_requests AS d 
  CROSS APPLY sys.fn_PageResCracker(d.page_resource) AS r
  CROSS APPLY sys.dm_db_page_info(r.db_id, r.file_id, r.page_id,'DETAILED')
    AS page_info;
GO

DBCC PAGE (2,1,116,3) WITH TABLERESULTS;

Commands completed successfully.

(49 rows affected)

(520 rows affected)

DBCC execution completed. If DBCC printed error messages, contact your system administrator.

Total execution time: 00:00:00.220

object_name,wait_type,database_id,file_id,page_id,page_header_version,page_type,page_type_desc,page_type_flag_bits,page_type_flag_bits_desc,page_flag_bits,page_flag_bits_desc,page_lsn,page_level,object_id,index_id,partition_id,alloc_unit_id,is_encrypted,has_checksum,checksum,is_iam_page,is_mixed_extent,has_ghost_records,has_version_records,pfs_page_id,pfs_is_allocated,pfs_alloc_percent,pfs_status,pfs_status_desc,gam_page_id,gam_status,gam_status_desc,sgam_page_id,sgam_status,sgam_status_desc,diff_map_page_id,diff_status,diff_status_desc,ml_map_page_id,ml_status,ml_status_desc,prev_page_file_id,prev_page_page_id,next_page_file_id,next_page_page_id,fixed_length,slot_count,ghost_rec_count,free_bytes,free_bytes_offset,reserved_bytes,reserved_bytes_by_xdes_id,xdes_id
sysschobjs,NULL,2,1,116,1,1,DATA_PAGE,0x4,,0x0,,00000050:00000a60:0142,0,34,1,281474978938880,281474978938880,0,0,1750484708,0,1,0,0,1,1,0,0x60,MIXED_EXT | PFS_IS_ALLOCATED | 0_PCT_FULL,2,1,ALLOCATED,3,0,NOT ALLOCATED,6,1,CHANGED,7,0,NOT MIN_LOGGED,0,0,1,300,44,19,0,5658,7578,0,0,0000:0000033c
sysschobjs,PAGELATCH_EX,2,1,116,1,1,DATA_PAGE,0x4,,0x0,,00000050:00000a60:0142,0,34,1,281474978938880,281474978938880,0,0,1750484708,0,1,0,0,1,1,0,0x60,MIXED_EXT | PFS_IS_ALLOCATED | 0_PCT_FULL,2,1,ALLOCATED,3,0,NOT ALLOCATED,6,1,CHANGED,7,0,NOT MIN_LOGGED,0,0,1,300,44,19,0,5658,7578,0,0,0000:0000033c
sysschobjs,PAGELATCH_EX,2,1,116,1,1,DATA_PAGE,0x4,,0x0,,00000050:00000a60:0142,0,34,1,281474978938880,281474978938880,0,0,1750484708,0,1,0,0,1,1,0,0x60,MIXED_EXT | PFS_IS_ALLOCATED | 0_PCT_FULL,2,1,ALLOCATED,3,0,NOT ALLOCATED,6,1,CHANGED,7,0,NOT MIN_LOGGED,0,0,1,300,44,19,0,5658,7578,0,0,0000:0000033c
sysschobjs,PAGELATCH_SH,2,1,116,1,1,DATA_PAGE,0x4,,0x0,,00000050:00000a60:0142,0,34,1,281474978938880,281474978938880,0,0,1750484708,0,1,0,0,1,1,0,0x60,MIXED_EXT | PFS_IS_ALLOCATED | 0_PCT_FULL,2,1,ALLOCATED,3,0,NOT ALLOCATED,6,1,CHANGED,7,0,NOT MIN_LOGGED,0,0,1,300,44,19,0,5658,7578,0,0,0000:0000033c
sysschobjs,NULL,2,1,116,1,1,DATA_PAGE,0x4,,0x0,,00000050:00000a60:0142,0,34,1,281474978938880,281474978938880,0,0,1750484708,0,1,0,0,1,1,0,0x60,MIXED_EXT | PFS_IS_ALLOCATED | 0_PCT_FULL,2,1,ALLOCATED,3,0,NOT ALLOCATED,6,1,CHANGED,7,0,NOT MIN_LOGGED,0,0,1,300,44,19,0,5658,7578,0,0,0000:0000033c
sysschobjs,PAGELATCH_EX,2,1,116,1,1,DATA_PAGE,0x4,,0x0,,00000050:00000a60:0142,0,34,1,281474978938880,281474978938880,0,0,1750484708,0,1,0,0,1,1,0,0x60,MIXED_EXT | PFS_IS_ALLOCATED | 0_PCT_FULL,2,1,ALLOCATED,3,0,NOT ALLOCATED,6,1,CHANGED,7,0,NOT MIN_LOGGED,0,0,1,300,44,19,0,5658,7578,0,0,0000:0000033c
sysschobjs,PAGELATCH_EX,2,1,116,1,1,DATA_PAGE,0x4,,0x0,,00000050:00000a60:0142,0,34,1,281474978938880,281474978938880,0,0,1750484708,0,1,0,0,1,1,0,0x60,MIXED_EXT | PFS_IS_ALLOCATED | 0_PCT_FULL,2,1,ALLOCATED,3,0,NOT ALLOCATED,6,1,CHANGED,7,0,NOT MIN_LOGGED,0,0,1,300,44,19,0,5658,7578,0,0,0000:0000033c
sysschobjs,PAGELATCH_EX,2,1,118,1,2,INDEX_PAGE,0x4,,0x0,,00000050:00000ad8:0182,0,34,2,562949955649536,562949955649536,0,0,-1683698132,0,1,1,0,1,1,0,0x68,MIXED_EXT | PFS_HAS_GHOST_RECORDS | PFS_IS_ALLOCATED | 0_PCT_FULL,2,1,ALLOCATED,3,0,NOT ALLOCATED,6,1,CHANGED,7,0,NOT MIN_LOGGED,0,0,1,344,10,66,24,3710,7232,0,0,0000:0004bce5
sysschobjs,PAGELATCH_EX,2,1,118,1,2,INDEX_PAGE,0x4,,0x0,,00000050:00000ad8:0182,0,34,2,562949955649536,562949955649536,0,0,-1683698132,0,1,1,0,1,1,0,0x68,MIXED_EXT | PFS_HAS_GHOST_RECORDS | PFS_IS_ALLOCATED | 0_PCT_FULL,2,1,ALLOCATED,3,0,NOT ALLOCATED,6,1,CHANGED,7,0,NOT MIN_LOGGED,0,0,1,344,10,66,24,3710,7232,0,0,0000:0004bce5
sysschobjs,PAGELATCH_EX,2,1,118,1,2,INDEX_PAGE,0x4,,0x0,,00000050:00000ad8:0182,0,34,2,562949955649536,562949955649536,0,0,-1683698132,0,1,1,0,1,1,0,0x68,MIXED_EXT | PFS_HAS_GHOST_RECORDS | PFS_IS_ALLOCATED | 0_PCT_FULL,2,1,ALLOCATED,3,0,NOT ALLOCATED,6,1,CHANGED,7,0,NOT MIN_LOGGED,0,0,1,344,10,66,24,3710,7232,0,0,0000:0004bce5


ParentObject,Object,Field,VALUE
BUFFER:,BUF @0x0000000B0EEA2C40,bpage,0x0000001016508000
BUFFER:,BUF @0x0000000B0EEA2C40,bPmmpage,0x0000000000000000
BUFFER:,BUF @0x0000000B0EEA2C40,bsort_r_nextbP,0x0000000000000000
BUFFER:,BUF @0x0000000B0EEA2C40,bsort_r_prevbP,0x0000000000000000
BUFFER:,BUF @0x0000000B0EEA2C40,bhash,0x0000000000000000
BUFFER:,BUF @0x0000000B0EEA2C40,bpageno,(1:116)
BUFFER:,BUF @0x0000000B0EEA2C40,bpart,0
BUFFER:,BUF @0x0000000B0EEA2C40,ckptGen,0x0000000000000000
BUFFER:,BUF @0x0000000B0EEA2C40,bDirtyRefCount,0
BUFFER:,BUF @0x0000000B0EEA2C40,bstat,0x10b


## Step 4 - Once the ostress utility completes, run the optimizetempdb.cmd file to enable im-memory tempdb system tables
### optimizetempdb.cmd contains the following code
### Change the user name, password, and server name in the cmd file as required
`sqlcmd -Usa -PJu@nS0t022 -ioptimizetempdb.sql -Slocalhost,1445
docker stop sqlopttempdb
docker start sqlopttempdb`
### Once changes are made, execute optimizetempdb.cmd by running the following code at the command line
`.\optimizetempdb.cmd`

## Step 5 - Execute tempstress.cmd again from the command line in the directory that contains the tempdb scripts
### The file contains the following code:
### Change the user name, password, and database as needed
`ostress -Usa -PJu@nS0t022 -Q"exec letsgosox" -n50 -r5000 -dChicagoWhiteSox -S"localhost,1445"`

## Step 6 - Run the query from Step 3 
### You should not receive any results

In [16]:
USE tempdb;
GO
SELECT object_name(page_info.object_id), d.wait_type, page_info.* 
FROM sys.dm_exec_requests AS d 
  CROSS APPLY sys.fn_PageResCracker(d.page_resource) AS r
  CROSS APPLY sys.dm_db_page_info(r.db_id, r.file_id, r.page_id,'DETAILED')
    AS page_info;
GO

Commands completed successfully.

(0 rows affected)

Total execution time: 00:00:00.217

(No column name),wait_type,database_id,file_id,page_id,page_header_version,page_type,page_type_desc,page_type_flag_bits,page_type_flag_bits_desc,page_flag_bits,page_flag_bits_desc,page_lsn,page_level,object_id,index_id,partition_id,alloc_unit_id,is_encrypted,has_checksum,checksum,is_iam_page,is_mixed_extent,has_ghost_records,has_version_records,pfs_page_id,pfs_is_allocated,pfs_alloc_percent,pfs_status,pfs_status_desc,gam_page_id,gam_status,gam_status_desc,sgam_page_id,sgam_status,sgam_status_desc,diff_map_page_id,diff_status,diff_status_desc,ml_map_page_id,ml_status,ml_status_desc,prev_page_file_id,prev_page_page_id,next_page_file_id,next_page_page_id,fixed_length,slot_count,ghost_rec_count,free_bytes,free_bytes_offset,reserved_bytes,reserved_bytes_by_xdes_id,xdes_id


## Step 7 - Find a list of memory-optimized objects
### Run the following query to return the memory optimized objects in tempdb

In [17]:
USE tempdb;
go
SELECT OBJECT_NAME(object_id), * FROM sys.dm_db_xtp_object_stats;
go

Commands completed successfully.

(10 rows affected)

Total execution time: 00:00:00.022

(No column name),object_id,xtp_object_id,row_insert_attempts,row_update_attempts,row_delete_attempts,write_conflicts,unique_constraint_violations,object_address
sysrscols,3,-2147483648,1376,0,34,0,0,0x0000000F1720A040
sysseobjvalues,9,-2147483647,0,0,0,0,0,0x0000000F17218040
sysschobjs,34,-2147483644,2570,1000054,4,0,0,0x0000000F1725A040
sysmultiobjvalues,40,-2147483643,0,0,0,0,0,0x0000000F16692040
syscolpars,41,-2147483640,1128,1,16,0,0,0x0000000F166E4040
sysidxstats,54,-2147483639,213,7,7,0,0,0x0000000F16802040
sysiscols,55,-2147483638,517,0,9,0,0,0x0000000F16820040
sysobjvalues,60,-2147483637,192,26,5,0,0,0x0000000F1683C040
syssingleobjrefs,74,-2147483634,205,0,4,0,0,0x0000000F1687E040
sysmultiobjrefs,75,-2147483633,107,0,0,0,0,0x0000000F16A1A040
